In [1]:
%reload_ext autoreload
%autoreload 2

import sys

sys.path.append('../Customized_Components') #this path is not right for this PC. Maybne this will work?
from bandaged_dolan import DolanJunctionBandage
from houcklab_qubit import DiffTransmonRounded

import warnings

import shapely
from CoupledLineTee import CoupledLineTee
from dolan_junction import DolanJunction
from LaunchpadWirebondCustom import LaunchpadWirebondCustom
from shapely.errors import ShapelyDeprecationWarning
from single_pad_transmon_pocket import TransmonPocket_Single

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)
from collections import OrderedDict

import Default_res_params as dp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import copy

# from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
# from qiskit_metal.qlibrary.terminations.launchpad_wb import LaunchpadWirebond
import pyEPR as epr
from qiskit_metal import Dict, Headings, MetalGUI, designs, draw
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.anchored_path import RouteAnchors
from qiskit_metal.qlibrary.tlines.mixed_path import RouteMeander, RouteMixed
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.toolbox_metal.parsing import *


In [2]:
gui = MetalGUI(dp.design)
design = dp.design
design.overwrite_enabled = True

09:26PM 35s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: Populating font family aliases took 178 ms. Replace uses of missing font family "Courier" with one that exists to avoid this cost. 



In [3]:
default_options = Dict(
        cut_l='1000um',
        cut_h='1200um',
        gap='55um',
        w = '90um',
        l = '900um',
        r = '60um',
        cpw_l = '50um',
        coupling_gap = '30um',
        JJ_cutout_w = '70um',
        JJ_cutout_h = '70um',
        JJ_cutout_r = '150um',
        JJ_c_contact_l = '40um',
        JJ_c_contact_r = '2.5um',
        JJ_c_contact_w = '10um',
        coupling_d = '200um',
        coupling_pad_w = '90um',
        coupling_r = '40um',
        cpw_pin = '10um',
        chip='main',
        resolution = '10',
        junction = 'False',
        orientation = '0',
        JJ_c_contact_shortl = '10um',
        istunnel = 'False')

2024-11-11 21:26:35.833 python[80176:4306112] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-11 21:26:35.833 python[80176:4306112] +[IMKInputSession subclass]: chose IMKInputSession_Modern


## Calculate Qubit Params

In [2]:
import Transmon_property as tp

In [3]:
import scqubits as scq
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as c

from scipy.optimize import root_scalar

In [4]:
ratio = 65
Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

def inductance_to_cap(L,ratio = ratio):
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    return (c.e.si**2 / (2 * EC)/c.h).to(u.fF)

def target_function(L, desired_frequency, ratio):
    L = L * u.nH  # Convert L to nanohenrys
    calculated_frequency = calculate_qubit_freq(L, ratio)
    return calculated_frequency - desired_frequency
def calculate_qubit_freq(L,ratio = ratio):
    
    EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
    EC = EJ/ratio
    ng = 0.0   # Offset charge
    nlev = 10  # Number of energy levels to consider

    # Create a Transmon object
    transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

    # Calculate the eigenenergies
    eigenenergies = transmon.eigenvals()

    # Calculate the transition frequencies (in GHz)
    transition_frequencies = np.diff(eigenenergies)

    # Calculate the anharmonicity (in GHz)
    anharmonicity = transition_frequencies[1] - transition_frequencies[0]
    return transition_frequencies[0]
    
class TransmonQubit():
    def __init__(self, freq, ratio):
        self.freq = freq
        self.ratio = ratio
        self.C = None,
        self.L = None
        self.EJ = None
        self.EC = None
        self.update_params()
        self.alpha = self.EC
        
    def update_params(self):
        self.C, self.L = self.qubit_freq_to_cap()
        Phi0 = (c.h / (2 * c.e.si))  # Flux quantum in Weber

        # Calculate the Josephson energy EJ and charging energy EC
        EJ = (((Phi0/2/np.pi)**2/self.L).to(u.J)/c.h).to(u.GHz)#, equivalencies=u.spectral())  # Convert to GHz
        EC = (c.e.si**2 / (2 * self.C)/c.h).to(u.GHz)
        self.EJ = EJ
        self.EC = EC
        
    def get_params(self):
        return [self.freq, self.ratio, self.qubit_capacitance, self.qubit_inductance, self.EJ, self.EC]
    def get_qcap(self):
        return self.C
    
    def get_qind(self):
        return self.L
    
    def get_EJ(self):
        return self.EJ
    
    def get_EC(self):
        return self.EC
    
    def qubit_freq_to_cap(self, ):
        desired_frequency = self.freq   # Desired frequency in GHz
        ratio = self.ratio
        # Use scipy.optimize.root_scalar to find the root
        def target_function(L, desired_frequency,):
            self.L = L * u.nH  # Convert L to nanohenrys
            self.calculate_qubit_freq()
            calculated_frequency = self.freq.to(u.GHz).value
            return calculated_frequency - desired_frequency
        result = root_scalar(target_function, args=(desired_frequency.to(u.GHz).value,), bracket=[1, 100], method='brentq')

        # Print the result
        if result.converged:
            L_optimal = result.root
            print(f"Optimal inductance L: {L_optimal} nH")
        else:
            print("Root finding did not converge")
        c_optimal = inductance_to_cap(L_optimal*u.nH, ratio = ratio)
        return c_optimal, L_optimal*u.nH
    def calculate_qubit_freq(self,):
        L = self.L
        ratio = self.ratio
        EJ = (((Phi0/2/np.pi)**2/L).to(u.J)/c.h).to(u.GHz)
        #, equivalencies=u.spectral())  # Convert to GHz
        EC = EJ/ratio
        ng = 0.0   # Offset charge
        nlev = 10  # Number of energy levels to consider

        # Create a Transmon object
        transmon = scq.Transmon(EJ=EJ.value, EC=EC.value, ng=ng, ncut=30, truncated_dim=nlev)

        # Calculate the eigenenergies
        eigenenergies = transmon.eigenvals()

        # Calculate the transition frequencies (in GHz)
        transition_frequencies = np.diff(eigenenergies)

        # Calculate the anharmonicity (in GHz)
        anharmonicity = transition_frequencies[1] - transition_frequencies[0]
        self.freq = transition_frequencies[0]*u.GHz
        self.alpha = anharmonicity*u.GHz
    

In [5]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)

Optimal inductance L: 13.676453476080171 nH


In [6]:
qubit.EC

<Quantity 0.18387753 GHz>

In [7]:
qubit.alpha

<Quantity 0.18387753 GHz>

Design specifics

In [141]:
from Transmon_specifications import find_junction_area, find_junction_capacitance

In [142]:
def pad_len_from_C(c):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*c.to(u.fF).value + intercept 
def unitless_find_j_area(L):
    return find_junction_area(L*u.nH,Jc = 0.42*u.uA/u.um**2).to(u.um**2).value

def unitless_find_pad_len(C):
    slope = (900-500)/(101.367-61.365)
    intercept = 500 - slope*61.365
    
    return slope*C + intercept

In [143]:
def convert_capacitance_to_energy(capacitance):
    C = capacitance*u.fF
    return (c.e.si**2 / (2 * C)/c.h).to(u.GHz)

def get_cap_params(caps, cj):
    cap_matrix = copy.deepcopy(caps)
    T = np.array([[1,0,-1,0],
              [0,1,0,0],
              [1,0,1,0],
              [0,0,0,1]])
    cap_matrix[0,2] -= cj
    cap_matrix[2,0] -= cj
    cap_matrix[2,2] += cj
    cap_matrix[0,0] += cj
    differential_cap_matrix = np.linalg.inv(T).T@(cap_matrix)@np.linalg.inv(T)
    inverse = np.linalg.inv(differential_cap_matrix)
    qubit_capacitance = np.reciprocal(inverse)[0,0]
    coupling_capacitance = np.reciprocal(inverse)[0,1]
    Ec = convert_capacitance_to_energy(qubit_capacitance)
    g = 8*convert_capacitance_to_energy(coupling_capacitance).to(u.MHz)
    return [Ec,g,qubit_capacitance,coupling_capacitance]
    

In [146]:
get_cap_params(np.array([[80,0,0,0],[0,80,0,0],[0,0,80,0],[0,0,0,80]]),0.5)

[<Quantity 0.48425573 GHz>, <Quantity 0. MHz>, 40.0, inf]

## Resonator calculations

In [8]:
import astropy.units as u
import astropy.constants as c
import scipy.optimize as optimize

# define constants for LL designs
sub_t = 350*u.um #substrate thickness (Si in this case)
metal_t = 100*u.nm #Deposited metal thickness (Al)
Sc = 67*u.fF/(u.um)**2 #JJ specific capacitance
epsilon = 11.45
W_jj = 200*u.nm #junction width
Z0 = 50*u.Ohm #characteristic impedance
import qiskit_metal.analyses as analyses

In [226]:
freq = 6*u.GHz.si
line_width = 15*u.um
line_gap = 8.30*u.um

Optimal pin-gap ratio for resonator and feedlines

In [9]:
class ReadoutResonator():
    def __init__(self, freq, short_on_one_end = False,optimal_pin = None, optimal_gap = None ):
        self.freq = freq
        self.short_on_one_end = short_on_one_end
        self.substrate_thickness = 350*u.um
        self.film_thickness = 100*u.nm
        if (optimal_gap is None) and (optimal_pin is None):
            self.pin = 15*u.um
            self.optimize_gap()
        elif optimal_gap is None:
            self.pin = optimal_pin
            self.optimize_gap()
        elif optimal_pin is None:
            self.gap = optimal_gap
            self.optimize_pin()
        
        self.param_from_res_freq(freq, short_on_one_end = short_on_one_end)
        
    def get_params(self):
        return [self.L, self.C, self.target_length]
    
    def get_L(self):
        return self.L
    
    def get_C(self):
        return self.C
    
    def get_target_length(self):
        return self.target_length
    
    def param_from_res_freq(self,freq, short_on_one_end = False):
        self.Lk, self.Lext,self.C,self.G,self.Z0,self.etf,self.Cstar = analyses.cpw_calculations.lumped_cpw(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)
        self.Lk*=u.H
        self.Lext*=u.H
        self.C*=u.F
        self.G*=u.S
        self.Z0*=u.Ohm
        self.etf*=u.H/u.m
        self.Cstar*=u.F
        target_length = analyses.cpw_calculations.guided_wavelength(freq = self.freq.to(u.Hz).value,
                                        line_width=self.pin.to(u.m).value,
                                        line_gap=self.gap.to(u.m).value,
                                        substrate_thickness=self.substrate_thickness.to(u.m).value,
                                        film_thickness=self.film_thickness.to(u.m).value,)[0]*u.m
        if short_on_one_end:
            target_length /= 4
        else:
            target_length /= 2
        self.len = target_length
    def optimize_gap(self):
        def f(line_gap):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            self.pin.to(u.m).value,
                                            line_gap,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_gap = optimize.fsolve(f,self.pin.si.value)[0]*u.m
        self.gap = optimized_gap.to(u.um)
    def optimize_pin(self):
        def f(line_width):
            return(analyses.cpw_calculations.lumped_cpw(self.freq.to(u.Hz).value,
                                            line_width,
                                            self.gap.si.value,
                                            self.substrate_thickness.to(u.m).value,
                                            self.film_thickness.to(u.m).value)[4]-50)
        optimized_pin = optimize.fsolve(f,self.gap.si.value)[0]*u.m
        self.pin = optimized_pin.to(u.um)

In [10]:
res = ReadoutResonator(6*u.GHz, short_on_one_end = False, optimal_pin = 20*u.um)

In [15]:
class DispersiveReadout():
    def __init__(self, qubit, res, T1, c_qr = None, c_tr = None):
        self.qubit = qubit
        self.res = res
        self.pT1 = T1
        self.optimize_cqr()
        self.optimize_ctr()
        
    def get_params(self):
        return [self.qubit, self.res, self.c_qr, self.c_tr]
    
    def get_cqr(self):
        return self.c_qr
    
    def get_ctr(self):
        return self.c_tr
    
    def get_qubit(self):
        return self.qubit
    
    def get_res(self):
        return self.res
    def calc_g(self):
        c_qr = self.c_qr
        Cq = self.qubit.C
        Cr = self.res.C
        wq = self.qubit.freq*np.pi*2
        wr = self.res.freq*np.pi*2
        EJ = self.qubit.EJ
        EC = self.qubit.EC
        return c_qr/Cq*np.sqrt(c.e.si**2*wr/c.hbar/Cr)*(EJ/8/EC)**0.25
    def calc_kappa(self):
        res = self.res
        c_tr = self.c_tr
        return (analyses.em.kappa_calculation.kappa_in(res.freq.si.value,
                                       c_tr.si.value,
                                       res.freq.si.value)*u.Hz).to(u.MHz)
    def calc_chi(self):
        qubit = self.qubit
        resonator = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-resonator.freq)*2*np.pi
        sum_freq = (qubit.freq+resonator.freq)*2*np.pi
        alpha = qubit.EC
        return 2*g_val**2*(alpha/delta/(delta+alpha)+alpha/sum_freq/(sum_freq+alpha))
    def calc_purcellT1(self,):
        qubit = self.qubit
        res = self.res
        kappa_val = self.calc_kappa()
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def calc_purcellT1q(self,):
        '''taking kappa = 2*chi for the optimal SNR, can directly get the T1q'''
        c_qr = self.c_qr
        qubit = self.qubit
        res = self.res
        g_val = self.calc_g()
        delta = (qubit.freq-res.freq)*2*np.pi
        
        chi_val = self.calc_chi()
        kappa_val = chi_val*2
        
        purcellGamma = g_val**2*kappa_val/delta**2
        
        return 1/purcellGamma/2/np.pi
    def optimize_cqr(self,):
        '''find the optimal coupling capacitance'''
        target_t1 = self.pT1
        c_qr = 1*u.fF
        self.c_qr = c_qr
        while self.calc_purcellT1q().to(u.ms) > 1.01*target_t1:
            c_qr += .1*u.fF
            self.c_qr = c_qr
        return c_qr
    def optimize_ctr(self, target_kappa=None):
        '''find the optimal coupling capacitance'''
        if target_kappa is None:
            target_kappa = self.calc_chi()*2
        c_tr = 1*u.fF
        self.c_tr = c_tr
        while self.calc_kappa() < target_kappa:
            c_tr += .1*u.fF
            self.c_tr = c_tr
        # return c_tr


In [16]:
qubit = TransmonQubit(4*u.GHz, ratio = 65)
res = ReadoutResonator(6*u.GHz, short_on_one_end = False)
dr = DispersiveReadout(qubit,res,5*u.ms)

Optimal inductance L: 13.676453476080171 nH


In [17]:
dr.calc_g().to(u.MHz)

<Quantity 178.66177702 MHz>

In [18]:
dr.calc_chi().to(u.MHz)

<Quantity 0.07840519 MHz>

In [19]:
dr.c_tr

<Quantity 21.4 fF>

In [478]:
import numpy as np
import matplotlib.pyplot as plt

def mandelbrot(width, height, x_min=-2.0, x_max=1.0, y_min=-1.5, y_max=1.5, max_iter=100):
    """
    Generate the Mandelbrot set and return it as a 2D array.

    Parameters:
        width (int): Width of the output array (number of pixels in the x direction).
        height (int): Height of the output array (number of pixels in the y direction).
        x_min (float): Minimum x-coordinate of the view.
        x_max (float): Maximum x-coordinate of the view.
        y_min (float): Minimum y-coordinate of the view.
        y_max (float): Maximum y-coordinate of the view.
        max_iter (int): Maximum number of iterations to determine divergence.

    Returns:
        np.ndarray: 2D array representing the Mandelbrot set.
    """
    # Create an array to store the Mandelbrot set data
    mandelbrot_set = np.zeros((height, width))

    # Create a grid of complex numbers over the specified range
    x_values = np.linspace(x_min, x_max, width)
    y_values = np.linspace(y_min, y_max, height)

    for i, y in enumerate(y_values):
        for j, x in enumerate(x_values):
            c = complex(x, y)
            z = 0
            iteration = 0

            # Iterate to determine if the point is in the Mandelbrot set
            while abs(z) <= 2 and iteration < max_iter:
                z = z*z + c
                iteration += 1

            # Store the iteration count (color intensity)
            mandelbrot_set[i, j] = iteration

    return mandelbrot_set

# Plotting function for the Mandelbrot set
def plot_mandelbrot(mandelbrot_set):
    plt.imshow(mandelbrot_set, cmap='twilight', extent=(-2, 1, -1.5, 1.5))
    plt.colorbar(label="Iteration Count")
    plt.title("Mandelbrot Set")
    plt.xlabel("Re(c)")
    plt.ylabel("Im(c)")
    plt.show()

# Generate and plot the Mandelbrot set
width, height = 1000, 1000
mandelbrot_set = mandelbrot(width, height)
plot_mandelbrot(mandelbrot_set)
